In [1]:
import json
import urllib.parse
import urllib.request
import requests

In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Modeling
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Capstone-KPMG/Modeling
'Copy of Geo_edited.ipynb'     gas_ev_charger_fast.csv
'Copy of Geo.ipynb'	       main_dataset_all_interstate_all_route.csv
'Copy of trafic_ratio.ipynb'   map_request_api.py
 Data			       num_EV_chargers_around_gas.ipynb
 gas_ev_charger_2.csv	       __pycache__
 gas_ev_charger_2.zip


In [3]:
from map_request_api import *

# main data set

In [4]:
gas = pd.read_csv('Data/main_dataset_all_interstate_all_route.csv')
gas

,gas_key,gas_name,gas_lat,gas_long,attr_cnt_1mile,attr_cnt_5mile,attr_name,attr_lat,attr_long,distance_to_nearest_attr,...,nri_risk_rating,nri_intpt_lat,nri_intpt_long,nri_zipcode,census_tract_area,census_tract_category,traff_cnt_10m_avg,traff_cnt_10m_max,traff_cnt_5m_avg,traff_cnt_5m_max
0,0,24 Hour Express Fuel,48.393760,-122.329806,0,1,Skagit County Fairgrounds,48.408145,-122.340118,1.1020,...,Relatively High,48.404815,-122.340554,98273.0,0.002191,2.0,21209.859155,72000,23733.802817,72000
1,1,76,47.191712,-120.916225,0,1,KBHR Set,47.223241,-120.993224,4.2247,...,NaN,NaN,NaN,NaN,0.000584,2.0,11678.727273,36000,9901.025641,34000
2,2,76,47.238704,-122.357577,0,4,Wagon Trail,47.275437,-122.308897,3.4174,...,Relatively High,47.249149,-122.364191,98424.0,0.001614,2.0,39766.900000,219000,43117.857143,219000
3,3,76,48.435227,-122.291324,0,1,Skagit County Fairgrounds,48.408145,-122.340118,2.9200,...,NaN,NaN,NaN,NaN,0.001533,2.0,20470.068493,72000,23101.204819,72000
4,4,76,47.839608,-120.014523,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.018331,0.0,5653.947368,12000,6723.076923,12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,2206,NaN,48.198460,-122.127847,0,1,Giant Arborvitae (Big Cedar Stump),48.168640,-122.189179,3.5008,...,Relatively Moderate,48.187437,-122.166077,98292.0,0.001817,2.0,26655.473684,123000,26604.166667,102000
2207,2207,NaN,48.556192,-122.344545,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.011283,0.0,19216.976744,72000,17606.250000,49000
2208,2208,NaN,48.747436,-122.475199,1,1,Mindport Exhibits,48.750881,-122.480674,0.3452,...,Relatively High,48.756483,-122.468299,98225.0,0.000309,2.0,21385.658915,69000,23751.250000,69000
2209,2209,NaN,48.755411,-122.463930,1,1,Mindport Exhibits,48.750881,-122.480674,0.8254,...,Relatively High,48.756483,-122.468299,98225.0,0.000309,2.0,21216.796875,69000,23464.634146,69000


# For each exit, find number of EV stations and sum of level 2 and fast charger numbers

In [5]:
# + ' ' + str(int(r_data['EV_DC_Fast_Count']))
def getRemoteEvDataList(df,lt,lg,name):

    df['shapePoints'] = df.apply(lambda x: [x[lt],x[lg]],axis=1)
    remoteDataList = []
    for row in range(df.shape[0]):
        inner = {}
        r_data = df.iloc[row]
        inner['name'] = str(int(r_data['EV_Level2_EVSE_Num'])) 
        inner['key'] = str(row)
        inner['shapePoints'] = r_data['shapePoints']
        remoteDataList.append(inner)
    return remoteDataList

# fill missing number of chargers to 0

In [6]:
ev_station = pd.read_csv('Data/alt_fuel_stations(Sep_25_2022).csv')
ev_station[['EV_Level2_EVSE_Num', 'EV_DC_Fast_Count']] = ev_station[['EV_Level2_EVSE_Num','EV_DC_Fast_Count']].fillna(value=0)
remoteDataListEv = getRemoteEvDataList(ev_station,'Latitude','Longitude','Station_Name')

In [7]:
# def get_avg_from_result(result):
#     filtered = pd.DataFrame(result['searchResults'])
#     #print(filtered['name'].astype(int).mean())
#     return filtered['key'], filtered['shapePoints'],  filtered['name'].astype(int).mean()

# def get_max_from_result(result):
#     filtered = pd.DataFrame(result['searchResults'])
#     #print(filtered['name'].astype(int).mean())
#     return filtered['key'], filtered['shapePoints'],  filtered['name'].astype(int).max()

def get_sum_from_result(result):
    filtered = pd.DataFrame(result['searchResults'])
    #print(filtered['name'].astype(int).mean())
    return filtered['key'], filtered['shapePoints'],  filtered['name'].astype(int).sum()

In [8]:
# for each gas, compute its total EV Stations and total chargers
gas_ev_data = []
for gas_i in range(gas.shape[0]):
  gas_data = gas.iloc[gas_i]
  gas_origin = [gas_data['gas_lat'], gas_data['gas_long']]

  # # 1 mile radius ev station and ev charger count
  # result_r1 = post_radius_search_request(gas_origin, remoteDataListEv, 1)

  # if result_r1['resultsCount'] != 0:
  #   ev_station_r1_count = result_r1['resultsCount']
  #   ev_name_1, ev_shape_1, ev_chargers_total_cnt_1 = get_sum_from_result(result_r1)

  # else:
  #   ev_station_r1_count = 0
  #   ev_chargers_total_cnt_1 = 0

  # # 2 mile radius ev station and ev charger count
  # result_r2 = post_radius_search_request(gas_origin, remoteDataListEv, 2)
  # if result_r2['resultsCount'] != 0:
  #   ev_station_r2_count = result_r2['resultsCount']
  #   ev_name_2, ev_shape_2, ev_chargers_total_cnt_2 = get_chargers(result_r2)

  # else:
  #   ev_station_r2_count = 0
  #   ev_chargers_total_cnt_2 = 0

  # 5 mile radius ev station and ev charger count
  result_r5 = post_radius_search_request(gas_origin, remoteDataListEv, 5)
  if result_r5['resultsCount'] != 0:
    ev_station_r5_count = result_r5['resultsCount']
    ev_name_5, ev_shape_5, ev_chargers_total_cnt_5 = get_sum_from_result(result_r5)
  else:
    ev_station_r5_count = 0
    ev_chargers_total_cnt_5 = 0

  # print(result_r5)

  
  gas_ev_data.append([gas_data['gas_key'], gas_data['gas_lat'], gas_data['gas_long'], 
                       ev_station_r5_count, ev_chargers_total_cnt_5])

In [11]:
gas_ev_data_df = pd.DataFrame(gas_ev_data)
gas_ev_data_df.columns = ['gas_key','gas_lat','gas_long','num_ev_station_cnt_5mile','num_ev_2_charger_cnt_5mile']
gas_ev_data_df

,gas_key,gas_lat,gas_long,num_ev_station_cnt_5mile,num_ev_2_charger_cnt_5mile
0,0,48.393760,-122.329806,14,24
1,1,47.191712,-120.916225,4,4
2,2,47.238704,-122.357577,49,94
3,3,48.435227,-122.291324,14,24
4,4,47.839608,-120.014523,4,5
...,...,...,...,...,...
2206,2206,48.198460,-122.127847,5,8
2207,2207,48.556192,-122.344545,1,0
2208,2208,48.747436,-122.475199,27,63
2209,2209,48.755411,-122.463930,27,63


In [12]:
gas_ev_data_df.to_csv('gas_ev_charger_2_2211.csv')

In [13]:
def getRemoteEvDataListFast(df,lt,lg,name):

    df['shapePoints'] = df.apply(lambda x: [x[lt],x[lg]],axis=1)
    remoteDataList = []
    for row in range(df.shape[0]):
        inner = {}
        r_data = df.iloc[row]
        inner['name'] = str(int(r_data['EV_DC_Fast_Count'])) 
        inner['key'] = str(row)
        inner['shapePoints'] = r_data['shapePoints']
        remoteDataList.append(inner)
    return remoteDataList

In [14]:
ev_station = pd.read_csv('Data/alt_fuel_stations(Sep_25_2022).csv')
ev_station[['EV_Level2_EVSE_Num', 'EV_DC_Fast_Count']] = ev_station[['EV_Level2_EVSE_Num','EV_DC_Fast_Count']].fillna(value=0)
remoteDataListEvFast = getRemoteEvDataListFast(ev_station,'Latitude','Longitude','Station_Name')

In [15]:
# for each gas, compute its total EV Stations and total chargers
gas_ev_data = []
for gas_i in range(gas.shape[0]):
  gas_data = gas.iloc[gas_i]
  gas_origin = [gas_data['gas_lat'], gas_data['gas_long']]

  # # 1 mile radius ev station and ev charger count
  # result_r1 = post_radius_search_request(gas_origin, remoteDataListEv, 1)

  # if result_r1['resultsCount'] != 0:
  #   ev_station_r1_count = result_r1['resultsCount']
  #   ev_name_1, ev_shape_1, ev_chargers_total_cnt_1 = get_sum_from_result(result_r1)

  # else:
  #   ev_station_r1_count = 0
  #   ev_chargers_total_cnt_1 = 0

  # # 2 mile radius ev station and ev charger count
  # result_r2 = post_radius_search_request(gas_origin, remoteDataListEv, 2)
  # if result_r2['resultsCount'] != 0:
  #   ev_station_r2_count = result_r2['resultsCount']
  #   ev_name_2, ev_shape_2, ev_chargers_total_cnt_2 = get_chargers(result_r2)

  # else:
  #   ev_station_r2_count = 0
  #   ev_chargers_total_cnt_2 = 0

  # 5 mile radius ev station and ev charger count
  result_r5 = post_radius_search_request(gas_origin, remoteDataListEvFast, 5)
  if result_r5['resultsCount'] != 0:
    ev_station_r5_count = result_r5['resultsCount']
    ev_name_5, ev_shape_5, ev_chargers_total_cnt_5 = get_sum_from_result(result_r5)
  else:
    ev_station_r5_count = 0
    ev_chargers_total_cnt_5 = 0

  # print(result_r5)

  
  gas_ev_data.append([gas_data['gas_key'], gas_data['gas_lat'], gas_data['gas_long'], 
                       ev_station_r5_count, ev_chargers_total_cnt_5])

In [16]:
gas_ev_data_df = pd.DataFrame(gas_ev_data)
gas_ev_data_df.columns = ['gas_key','gas_lat','gas_long','num_ev_station_cnt_5mile','num_ev_fast_charger_cnt_5mile']
gas_ev_data_df

,gas_key,gas_lat,gas_long,num_ev_station_cnt_5mile,num_ev_fast_charger_cnt_5mile
0,0,48.393760,-122.329806,14,5
1,1,47.191712,-120.916225,4,14
2,2,47.238704,-122.357577,49,15
3,3,48.435227,-122.291324,14,5
4,4,47.839608,-120.014523,4,0
...,...,...,...,...,...
2206,2206,48.198460,-122.127847,5,16
2207,2207,48.556192,-122.344545,1,8
2208,2208,48.747436,-122.475199,27,15
2209,2209,48.755411,-122.463930,27,15


In [17]:
gas_ev_data_df.to_csv('gas_ev_charger_fast_2211.csv')

# combine two data frames

In [18]:
fast = pd.read_csv('gas_ev_charger_fast_2211.csv')
fast.columns

Index(['Unnamed: 0', 'gas_key', 'gas_lat', 'gas_long',
       'num_ev_station_cnt_5mile', 'num_ev_fast_charger_cnt_5mile'],
      dtype='object')

In [19]:
level_2 = pd.read_csv('gas_ev_charger_2_2211.csv')
level_2.columns

Index(['Unnamed: 0', 'gas_key', 'gas_lat', 'gas_long',
       'num_ev_station_cnt_5mile', 'num_ev_2_charger_cnt_5mile'],
      dtype='object')

In [21]:
new = fast.merge(level_2, left_on='gas_key', right_on='gas_key')

In [24]:
new.columns

Index(['Unnamed: 0_x', 'gas_key', 'gas_lat_x', 'gas_long_x',
       'num_ev_station_cnt_5mile_x', 'num_ev_fast_charger_cnt_5mile',
       'Unnamed: 0_y', 'gas_lat_y', 'gas_long_y', 'num_ev_station_cnt_5mile_y',
       'num_ev_2_charger_cnt_5mile'],
      dtype='object')

In [25]:
new = new[['gas_key', 'gas_lat_x', 'gas_long_x',
       'num_ev_station_cnt_5mile_x', 'num_ev_fast_charger_cnt_5mile','num_ev_2_charger_cnt_5mile']]
    

In [26]:
new

,gas_key,gas_lat_x,gas_long_x,num_ev_station_cnt_5mile_x,num_ev_fast_charger_cnt_5mile,num_ev_2_charger_cnt_5mile
0,0,48.393760,-122.329806,14,5,24
1,1,47.191712,-120.916225,4,14,4
2,2,47.238704,-122.357577,49,15,94
3,3,48.435227,-122.291324,14,5,24
4,4,47.839608,-120.014523,4,0,5
...,...,...,...,...,...,...
2206,2206,48.198460,-122.127847,5,16,8
2207,2207,48.556192,-122.344545,1,8,0
2208,2208,48.747436,-122.475199,27,15,63
2209,2209,48.755411,-122.463930,27,15,63


In [27]:
main_with_ntr = pd.read_csv('main_dataset_all_interstate_all_route.csv')
main_with_ntr

,gas_key,gas_name,gas_lat,gas_long,attr_cnt_1mile,attr_cnt_5mile,attr_name,attr_lat,attr_long,distance_to_nearest_attr,...,nri_risk_rating,nri_intpt_lat,nri_intpt_long,nri_zipcode,census_tract_area,census_tract_category,traff_cnt_10m_avg,traff_cnt_10m_max,traff_cnt_5m_avg,traff_cnt_5m_max
0,0,24 Hour Express Fuel,48.393760,-122.329806,0,1,Skagit County Fairgrounds,48.408145,-122.340118,1.1020,...,Relatively High,48.404815,-122.340554,98273.0,0.002191,2.0,21209.859155,72000,23733.802817,72000
1,1,76,47.191712,-120.916225,0,1,KBHR Set,47.223241,-120.993224,4.2247,...,NaN,NaN,NaN,NaN,0.000584,2.0,11678.727273,36000,9901.025641,34000
2,2,76,47.238704,-122.357577,0,4,Wagon Trail,47.275437,-122.308897,3.4174,...,Relatively High,47.249149,-122.364191,98424.0,0.001614,2.0,39766.900000,219000,43117.857143,219000
3,3,76,48.435227,-122.291324,0,1,Skagit County Fairgrounds,48.408145,-122.340118,2.9200,...,NaN,NaN,NaN,NaN,0.001533,2.0,20470.068493,72000,23101.204819,72000
4,4,76,47.839608,-120.014523,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.018331,0.0,5653.947368,12000,6723.076923,12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,2206,NaN,48.198460,-122.127847,0,1,Giant Arborvitae (Big Cedar Stump),48.168640,-122.189179,3.5008,...,Relatively Moderate,48.187437,-122.166077,98292.0,0.001817,2.0,26655.473684,123000,26604.166667,102000
2207,2207,NaN,48.556192,-122.344545,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.011283,0.0,19216.976744,72000,17606.250000,49000
2208,2208,NaN,48.747436,-122.475199,1,1,Mindport Exhibits,48.750881,-122.480674,0.3452,...,Relatively High,48.756483,-122.468299,98225.0,0.000309,2.0,21385.658915,69000,23751.250000,69000
2209,2209,NaN,48.755411,-122.463930,1,1,Mindport Exhibits,48.750881,-122.480674,0.8254,...,Relatively High,48.756483,-122.468299,98225.0,0.000309,2.0,21216.796875,69000,23464.634146,69000


In [28]:
new_main = main_with_ntr.merge(new, left_on='gas_key', right_on='gas_key')

In [29]:
new_main.columns

Index(['gas_key', 'gas_name', 'gas_lat', 'gas_long', 'attr_cnt_1mile',
       'attr_cnt_5mile', 'attr_name', 'attr_lat', 'attr_long',
       'distance_to_nearest_attr', 'crime_coord', 'crime_county',
       'crime_population', 'violent_crime', 'murder_nonnegligent_manslaughter',
       'Rape1', 'Robbery', 'aggravated_assault', 'property_crime', 'Burglary',
       'larceny_theft', 'motor_vehicle_theft', 'Arson', 'total_crime',
       'exit_name', 'exit_lat', 'exit_long', 'distance_to_nearest_exit',
       'exit', 'num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas',
       'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas',
       'num_EV_in_50_miles_of_gas', 'Closest_EV_Station_name',
       'Closest_EV_Station_lat', 'Closest_EV_Station_long',
       'distance_to_closest_ev_station', 'nri_geoid', 'nri_county',
       'nri_population', 'nri_build_value', 'nri_agri_value', 'nri_area',
       'nri_risk_score', 'nri_risk_rating', 'nri_intpt_lat', 'nri_intpt_long',
       'nri_zipco

In [32]:
new_main_final = new_main[['gas_key', 'gas_name', 'gas_lat', 'gas_long', 'attr_cnt_1mile',
       'attr_cnt_5mile', 'attr_name', 'attr_lat', 'attr_long',
       'distance_to_nearest_attr', 'crime_coord', 'crime_county',
       'crime_population', 'violent_crime', 'murder_nonnegligent_manslaughter',
       'Rape1', 'Robbery', 'aggravated_assault', 'property_crime', 'Burglary',
       'larceny_theft', 'motor_vehicle_theft', 'Arson', 'total_crime',
       'exit_name', 'exit_lat', 'exit_long', 'distance_to_nearest_exit',
       'exit', 'num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas',
       'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas',
       'num_EV_in_50_miles_of_gas', 'Closest_EV_Station_name',
       'Closest_EV_Station_lat', 'Closest_EV_Station_long',
       'distance_to_closest_ev_station', 'nri_geoid', 'nri_county',
       'nri_population', 'nri_build_value', 'nri_agri_value', 'nri_area',
       'nri_risk_score', 'nri_risk_rating', 'nri_intpt_lat', 'nri_intpt_long',
       'nri_zipcode', 'census_tract_area', 'census_tract_category',
       'traff_cnt_10m_avg', 'traff_cnt_10m_max', 'traff_cnt_5m_avg',
       'traff_cnt_5m_max', 'num_ev_fast_charger_cnt_5mile',
       'num_ev_2_charger_cnt_5mile']]

In [35]:
new_main_final

,gas_key,gas_name,gas_lat,gas_long,attr_cnt_1mile,attr_cnt_5mile,attr_name,attr_lat,attr_long,distance_to_nearest_attr,...,nri_intpt_long,nri_zipcode,census_tract_area,census_tract_category,traff_cnt_10m_avg,traff_cnt_10m_max,traff_cnt_5m_avg,traff_cnt_5m_max,num_ev_fast_charger_cnt_5mile,num_ev_2_charger_cnt_5mile
0,0,24 Hour Express Fuel,48.393760,-122.329806,0,1,Skagit County Fairgrounds,48.408145,-122.340118,1.1020,...,-122.340554,98273.0,0.002191,2.0,21209.859155,72000,23733.802817,72000,5,24
1,1,76,47.191712,-120.916225,0,1,KBHR Set,47.223241,-120.993224,4.2247,...,NaN,NaN,0.000584,2.0,11678.727273,36000,9901.025641,34000,14,4
2,2,76,47.238704,-122.357577,0,4,Wagon Trail,47.275437,-122.308897,3.4174,...,-122.364191,98424.0,0.001614,2.0,39766.900000,219000,43117.857143,219000,15,94
3,3,76,48.435227,-122.291324,0,1,Skagit County Fairgrounds,48.408145,-122.340118,2.9200,...,NaN,NaN,0.001533,2.0,20470.068493,72000,23101.204819,72000,5,24
4,4,76,47.839608,-120.014523,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.018331,0.0,5653.947368,12000,6723.076923,12000,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,2206,NaN,48.198460,-122.127847,0,1,Giant Arborvitae (Big Cedar Stump),48.168640,-122.189179,3.5008,...,-122.166077,98292.0,0.001817,2.0,26655.473684,123000,26604.166667,102000,16,8
2207,2207,NaN,48.556192,-122.344545,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.011283,0.0,19216.976744,72000,17606.250000,49000,8,0
2208,2208,NaN,48.747436,-122.475199,1,1,Mindport Exhibits,48.750881,-122.480674,0.3452,...,-122.468299,98225.0,0.000309,2.0,21385.658915,69000,23751.250000,69000,15,63
2209,2209,NaN,48.755411,-122.463930,1,1,Mindport Exhibits,48.750881,-122.480674,0.8254,...,-122.468299,98225.0,0.000309,2.0,21216.796875,69000,23464.634146,69000,15,63


In [34]:
new_main_final.columns = ['gas_key', 'gas_name', 'gas_lat', 'gas_long', 'attr_cnt_1mile',
       'attr_cnt_5mile', 'attr_name', 'attr_lat', 'attr_long',
       'distance_to_nearest_attr', 'crime_coord', 'crime_county',
       'crime_population', 'violent_crime', 'murder_nonnegligent_manslaughter',
       'Rape1', 'Robbery', 'aggravated_assault', 'property_crime', 'Burglary',
       'larceny_theft', 'motor_vehicle_theft', 'Arson', 'total_crime',
       'exit_name', 'exit_lat', 'exit_long', 'distance_to_nearest_exit',
       'highway', 'num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas',
       'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas',
       'num_EV_in_50_miles_of_gas', 'Closest_EV_Station_name',
       'Closest_EV_Station_lat', 'Closest_EV_Station_long',
       'distance_to_closest_ev_station', 'nri_geoid', 'nri_county',
       'nri_population', 'nri_build_value', 'nri_agri_value', 'nri_area',
       'nri_risk_score', 'nri_risk_rating', 'nri_intpt_lat', 'nri_intpt_long',
       'nri_zipcode', 'census_tract_area', 'census_tract_category',
       'traff_cnt_10m_avg', 'traff_cnt_10m_max', 'traff_cnt_5m_avg',
       'traff_cnt_5m_max', 'num_ev_fast_charger_cnt_5mile',
       'num_ev_2_charger_cnt_5mile']

In [39]:
new_main_final.to_csv('main_dataset_all_interstate_all_route_11_09.csv')